# FAISS (Facebook AI Similarity Search) 

It is a library developed by Facebook for efficient similarity search and clustering of dense vectors. In simple terms, it helps you quickly find items that are similar to each other in a large dataset.

Imagine you have a huge collection of photos, and you want to find photos that look similar to a specific one. FAISS can help you do that by comparing the features of the photos and quickly identifying the ones that are most similar. It's like having a smart assistant that can quickly find things that are alike in a big pile of data.

In [22]:
# Install the FAISS library for CPU (Facebook AI Similarity Search)
# This library is used for efficient similarity search and clustering of dense vectors
%pip install faiss-cpu

# Install the Sentence-Transformers library
# This library provides pre-trained models for converting sentences into dense vectors
# These vectors can then be used for tasks like similarity search
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [24]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [25]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [26]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


# Step 1 : Create source embeddings for the text column

In [27]:
from sentence_transformers import SentenceTransformer

In [28]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [29]:
vectors.shape

(8, 768)

In [30]:
vectors.shape[1]

768

In [31]:
vectors[1]

array([-3.35726663e-02,  9.80516616e-03, -3.25012915e-02,  1.86820552e-02,
        1.77040920e-02, -3.62627096e-02, -3.62390541e-02,  7.10840076e-02,
        2.47629788e-02,  2.63498593e-02, -1.21443206e-03, -1.13375476e-02,
       -1.15268650e-02,  1.43791335e-02, -4.38231789e-02, -2.79519018e-02,
        1.75678618e-02, -4.70076315e-03,  8.76717716e-02,  4.92516265e-04,
       -4.46608923e-02, -3.32105868e-02, -1.42118577e-02,  2.29377411e-02,
       -8.52665491e-03, -2.48491648e-03,  8.57958850e-03,  2.66635101e-02,
        2.85466053e-02, -2.94707753e-02, -4.25882787e-02,  1.92414727e-02,
       -2.21508741e-02, -5.22682518e-02,  9.17419413e-07, -2.53168028e-02,
       -5.45516051e-02,  6.14561178e-02, -1.15586128e-02,  1.64797101e-02,
        6.11980737e-04, -6.41633272e-02, -2.64592692e-02,  2.86634415e-02,
       -7.40527408e-03, -3.89426798e-02, -8.28326796e-04,  1.00291803e-01,
       -6.97314832e-03,  2.48221084e-02,  1.85312573e-02, -5.47105353e-03,
        2.40095705e-02, -

In [32]:
vectors

array([[-0.00247395,  0.03626721, -0.05290459, ..., -0.09152356,
        -0.03970002, -0.04330488],
       [-0.03357267,  0.00980517, -0.03250129, ..., -0.05165467,
         0.02245886, -0.03156182],
       [-0.01865323, -0.04051314, -0.01235387, ...,  0.00610585,
        -0.07179647,  0.02773853],
       ...,
       [-0.00066458,  0.04252125, -0.05645508, ...,  0.01315471,
        -0.03183568, -0.04357664],
       [-0.03317154,  0.03252455, -0.02484838, ...,  0.01174421,
         0.05747123,  0.00571021],
       [-0.00166395,  0.00413827, -0.04597082, ...,  0.02008527,
         0.05656242, -0.00161595]], dtype=float32)

In [33]:
dim = vectors.shape[1]
dim

768

# Step 2 : Build a FAISS Index for vectors

In [34]:
import faiss

index = faiss.IndexFlatL2(dim)

# Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [35]:
index.add(vectors)

In [36]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002AEDFA53CF0> >

# Step 4 : Encode search text using same encorder and normalize the output vector

In [37]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [38]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

# Step 5: Search for similar vector in the FAISS index created

In [42]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039094]], dtype=float32)

In [43]:
I

array([[3, 2]], dtype=int64)

In [45]:
df.loc[I[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [46]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query